# Reactive nitrogen
## Calculate the critical [NO]
Essentially when P(O$_3$) = L(O$_3$). We want to solve

\begin{equation}
[NO] = \frac{[O_3] \cdot ( k^1_l[OH] + k^2_l [HO_2] + f J_{O1D})}{k^1[HO_2] + k^2 [CH_3O_2]}.
\end{equation}

In [1]:
import math

In [2]:
# Conditions
p_sfc = 101325 # pressure
p_10 = 28000
R = 8.31 # 
NA = 6.022 * 1e23  # Avogrado
t_sfc = 298

# Concentrations
ho2 = 40 * 1e-12
ch3o2 = 25 * 1e-12
o3 = 40 * 1e-9
oh_c = 1e6 # Note: number density
f = 0.15
j_o1d = 2.5 * 1e-5

# Reaction rates
k1 = k2 = 8.5 * 1e-12
k1_l = 7.3 * 1e-14
k2_l = 2 * 1e-15

We need to convert the volume mixing ratios to number densities:

In [3]:
# Just a simple function to convert from volume mixing ratio
# to number density.
def vol_to_dens(vol, temp, pressure):
    return((vol * NA * pressure) / (R * temp * 1e6))


ho2_c = vol_to_dens(ho2, t_sfc, p_sfc)
ch3o2_c = vol_to_dens(ch3o2, t_sfc, p_sfc)
o3_c = vol_to_dens(o3, t_sfc, p_sfc)

We can now solve the equation for [NO].

In [4]:
no_crit = (o3_c * (k1_l * oh_c + k2_l * ho2_c + f * j_o1d)) /\
            (k1 * ho2_c + k2 * ch3o2_c)
print(f'The critical NO concentration is: {no_crit:.2e} molec cm^-3')

The critical NO concentration is: 4.19e+08 molec cm^-3


Let's have a look at the individual parts of the equation to see which part has the largest impact on the steady state.

In [5]:
print(f'k1_l * [OH]: {k1_l * oh_c:.2e}')
print(f'k2_l * [HO_2]: {k2_l * ho2_c:.2e}')
print(f'f * JO1D$: {f * j_o1d: .2e}')
print(f'k1 * ho2_c: {k1 * ho2_c:.2e}')
print(f'k2 * ch202_c: {k2 * ch3o2_c:.2e}')

k1_l * [OH]: 7.30e-08
k2_l * [HO_2]: 1.97e-06
f * JO1D$:  3.75e-06
k1 * ho2_c: 8.38e-03
k2 * ch202_c: 5.24e-03


As seen in the output above, $k^1 [HO_2]$ is the most important part of the critical [NO] since it is the fastest reaction and thus has the largest impact on the concentration.

## Calculation of lifetime of NO$_x$.

Calculating the lifetime of NO$_x$ is based on the following equation

\begin{equation}
\tau_{NO_x} = \tau_{NO_2} (1 + [NO] / [NO_2]).
\end{equation}

This is done at the surface and at 10km. We need to define the following conditions:

In [6]:
oh_c = 1e6
j_no2 = 1e-2
t_10 = 220
o3_sfc = 40 * 1e-9
o3_10 = 100 * 1e-9
# N2 volume mixing ratio in air. Altitude independent?
n2 = 78.08 * 1e-2
# Convert volume mixing ration to number dens.
o3_sfc_c = vol_to_dens(o3_sfc, t_sfc, p_sfc)
o3_10_c = vol_to_dens(o3_10, t_10, p_10)
n2_sfc_c = vol_to_dens(n2, t_sfc, p_sfc)
n2_10_c = vol_to_dens(n2, t_10, p_10)

# Reaction rates
k_sfc = 1.4 * 1e-12 * math.exp(-1310 / t_sfc) 
k_10 = 1.4 * 1e-12 * math.exp(-1310 / t_10)
kn_sfc = 3.3 * 1e-30 * (t_sfc / 300)**-3.0 * n2_sfc_c
kn_10 = 3.3 * 1e-30 * (t_10 / 300)**-3.0 * n2_10_c

The ratio of $[NO_2]/[NO]$ is calculated as

\begin{equation}
[NO] / [NO_2]  = \frac{J_{NO_2}}{k[O_3]}
\end{equation}

In [7]:
# Surface ratio
rat_sfc =  (j_no2) / (k_sfc * o3_sfc_c)
print(f'[NO]/[NO2] ratio at surface: {rat_sfc:.2f}')

[NO]/[NO2] ratio at surface: 0.59


In [8]:
# Ratio at 10 km
rat_10 = (j_no2) / (k_10 * o3_10_c)
print(f'[NO]/[NO2] ratio at 10 km: {rat_10:.2f}')

[NO]/[NO2] ratio at 10 km: 2.99


Furthermore, the lifetime of NO$_2$ can be calculated as

\begin{equation}
\tau_{NO_2} = \frac{1}{k_n [OH]}
\end{equation}

In [9]:
tau_no2_sfc = 1 / (kn_sfc * oh_c)
tau_no2_10 = 1 / (kn_10 * oh_c)

which is then used to calculate the lifetime of NO$_x$

In [10]:
tau_nox_sfc = tau_no2_sfc * (1 + rat_sfc)
tau_nox_10 = tau_no2_10 * (1 + rat_10)
print(f'Lifetime of NO_x at surface: {tau_nox_sfc/3600:.2f} hours.')
print(f'Lifetime of NO_x at 10 km: {tau_nox_10/3600:.2f} hours.')

Lifetime of NO_x at surface: 6.81 hours.
Lifetime of NO_x at 10 km: 18.37 hours.
